In [1]:
#ignore warning
import warnings
warnings.filterwarnings('ignore')

In [2]:
# import yfinance, pandas and os
import yfinance as yf
import pandas as pd
import datetime
import os
import yesg

In [12]:
# Tickers, Forbes Just 100 top esg companies with all financial and esg details are available on yahoo finance
Symbols = ["INTC", "MSFT", "BAC", "AAPL", "NVDA", "VZ", "CSCO", "EXC", "PEP", "MA", "HIG", "C", "ANTM", "ACN", "T", "IBM",
           "F", "OC", "INTU", "WFC", "MRK", "JPM", "AEP", "UNH", "CI", "AMAT", "CL", "EQIX", "UPS", "NEM", "LMT", "BLK",
           "EIX", "JNJ", "SYF", "FDX", "GS", "NOC", "PG", "ADM", "ETR", "QCOM", "GIS", "DE", "RF", "LOW", "AMGN", "ABT",
           "HPE", "BDX", "CLX", "AES", "SPGI", "ES", "HSY", "AXP", "NLSN", "EBAY", "ECL", "NDAQ", "DLR","COF", "BBY", 
           "KMB", "EMN"]

In [14]:
def create_esg_full(Symbols):
    esg_data = pd.DataFrame()
    
    for i in Symbols:
        try:
            temp =pd.DataFrame(yesg.get_esg_full(i))
            
            # append the individual stock info
            if len(temp) == 0:
                None
            else:
                esg_data = esg_data.append(temp, ignore_index = True) 
#                 print(f'{i}: esg appended')
        
        except Exception:
            None
    # Data cleaning to ensure table names are proper, no null or duplicate data and datatypes are correct     
    esg_data['Total-Score'] = pd.to_numeric(esg_data['Total-Score'])
    esg_data['E-Score'] = pd.to_numeric(esg_data['E-Score'])
    esg_data['S-Score'] = pd.to_numeric(esg_data['S-Score'])
    esg_data['G-Score'] = pd.to_numeric(esg_data['G-Score'])
    esg_data['Last Rated']= pd.to_datetime(esg_data['Last Rated'])
    esg_data.drop(['environment Percentile','social Percentile', 'governance Percentile'], axis = 1, inplace = True)
    esg_data.loc[esg_data['Controversial Business Areas'] == "", "Controversial Business Areas"] = 'None'
    esg_data.loc[esg_data['related Controversy'] == "-", "related Controversy"] = 'None'
    esg_data['ESG Performance'].fillna('Not Available', inplace = True)
    esg_data.insert(0, "Ticker", esg_data.pop('Ticker'))
    esg_data.rename(columns={'Last Rated': 'lastRated', 'ESG Performance': 'esgPerformance','peer Group': 'peerGroup',
                             "Highest Controversy" : "highestControversy", "peer Count": "peerCount",
                             "total Percentile":"totalPercentile", "related Controversy": "relatedControversy",
                             'min peer ESG': "minPeerESG", "avg peer ESG": "avgPeerESG", "max peer ESG":"maxPeerESG",
                             'min peer Environment': "minPeerEnvironment", "avg peer Environment": "avgPeerEnvironment",
                             "max peer Environment":"maxPeerEnvironment",'min peer Social': "minPeerSocial", 
                             "avg peer Social": "avgPeerSocial", "max peer Social":"maxPeerSocial",
                             'min peer Governance': "minPeerGovernance", "avg peer Governance": "avgPeerGovernance",
                             "max peer Governance":"maxPeerGovernance", "min Highest Controversy": "minHighestControversy",
                             "avg Highest Controversy": "avgHighestControversy", "max Highest Controversy": "maxHighestControversy",
                             "Controversial Business Areas" : "controversialBusinessAreas",'Total-Score': 'totalScore',
                             'E-Score': 'eScore', 'S-Score': 'sScore', 'G-Score': 'gScore'
                             }, inplace=True)
    
    
    return esg_data
    

In [15]:
esg_data = create_esg_full(Symbols)
print(f'total stocks in esg_full is : {len(esg_data)}')

INTC: esg appended
MSFT: esg appended
BAC: esg appended
AAPL: esg appended
NVDA: esg appended
VZ: esg appended
CSCO: esg appended
EXC: esg appended
PEP: esg appended
MA: esg appended
HIG: esg appended
C: esg appended
ANTM: esg appended
ACN: esg appended
T: esg appended
IBM: esg appended
F: esg appended
OC: esg appended
INTU: esg appended
WFC: esg appended
MRK: esg appended
JPM: esg appended
AEP: esg appended
UNH: esg appended
CI: esg appended
AMAT: esg appended
CL: esg appended
EQIX: esg appended
UPS: esg appended
NEM: esg appended
LMT: esg appended
BLK: esg appended
EIX: esg appended
JNJ: esg appended
SYF: esg appended
FDX: esg appended
GS: esg appended
NOC: esg appended
PG: esg appended
ADM: esg appended
ETR: esg appended
QCOM: esg appended
GIS: esg appended
DE: esg appended
RF: esg appended
LOW: esg appended
AMGN: esg appended
ABT: esg appended
HPE: esg appended
BDX: esg appended
CLX: esg appended
AES: esg appended
SPGI: esg appended
ES: esg appended
HSY: esg appended
AXP: esg appen

In [19]:
# Created a function to extract stocks description from yahoo finance
def create_info_list(Symbols):
    # create empty list
    info = []
    # iterate over each symbol
    for i in Symbols:  
        try:
            # download stock info
            temp = []
            temp = yf.Ticker(i).info
                        
            # append the individual stock info
            if len(temp) == 0:
                None
            else:
                info.append(temp)
#                 print(f'{i}: info appended')
                
        except Exception:
            None
   
    return info


In [20]:
def create_info(Symbols):
    info_df = pd.DataFrame()
    info = create_info_list(Symbols)
#     print(len(info))
#     print(len(Symbols))
    if len(info) == 0:
        None
    else:
        info_df = info_df.append(info,sort=False, ignore_index=True)
        
    # Data cleaning to ensure table names are proper, no null or duplicate data and datatypes are correct   
    info_df = info_df[['symbol','longName', 'sector','industry', 'city', 'state','country','fullTimeEmployees',
                       'revenueGrowth', 'forwardEps', 'beta' , 'priceToSalesTrailing12Months' , 'forwardPE', 
                       'dividendRate', 'trailingPE', 'bookValue', 'returnOnAssets', 'returnOnEquity','averageVolume', 'volume','currentPrice',
                       'fiftyTwoWeekLow','fiftyTwoWeekHigh', 'regularMarketPrice', 'marketCap']]
    info_df.rename(columns = {'symbol':'Ticker'}, inplace = True)
    info_df.loc[info_df['country'] == 'Ireland', 'state'] = 'Leinster'
    info_df['industry'] = esg_data['peerGroup']
    return info_df

In [21]:
info_df = create_info(esg_data['Ticker'].to_list())
print(f'total stocks in stock_details is : {len(info_df)}')

INTC: info appended
MSFT: info appended
BAC: info appended
AAPL: info appended
NVDA: info appended
VZ: info appended
CSCO: info appended
EXC: info appended
PEP: info appended
MA: info appended
HIG: info appended
C: info appended
ANTM: info appended
ACN: info appended
T: info appended
IBM: info appended
F: info appended
OC: info appended
INTU: info appended
WFC: info appended
MRK: info appended
JPM: info appended
AEP: info appended
UNH: info appended
CI: info appended
AMAT: info appended
CL: info appended
EQIX: info appended
UPS: info appended
NEM: info appended
LMT: info appended
BLK: info appended
EIX: info appended
JNJ: info appended
SYF: info appended
FDX: info appended
GS: info appended
NOC: info appended
PG: info appended
ADM: info appended
ETR: info appended
QCOM: info appended
GIS: info appended
DE: info appended
RF: info appended
LOW: info appended
AMGN: info appended
ABT: info appended
HPE: info appended
BDX: info appended
CLX: info appended
AES: info appended
SPGI: info appen

,Ticker,longName,sector,industry,city,state,country,fullTimeEmployees,revenueGrowth,forwardEps,beta,priceToSalesTrailing12Months,forwardPE,dividendRate,trailingPE,bookValue,returnOnAssets,returnOnEquity,averageVolume,volume,currentPrice,fiftyTwoWeekLow,fiftyTwoWeekHigh,regularMarketPrice,marketCap
0,INTC,Intel Corporation,Technology,Semiconductors,Santa Clara,CA,United States,121100,0.028,3.72,0.545767,2.670730,13.932797,1.46,10.664609,23.438,0.08586,0.22522,39453624,33249492,51.83,43.63,68.49,51.83,211051773952
1,MSFT,Microsoft Corporation,Technology,Software—Infrastructure,Redmond,WA,United States,181000,0.201,10.75,0.910925,12.312669,28.249302,2.48,32.344230,21.335,0.15249,0.49051,37450156,22566453,303.68,231.10,349.67,303.68,2276649336832
2,BAC,Bank of America Corporation,Financial Services,Banks—Diversified,Charlotte,NC,United States,208000,0.125,3.84,1.415846,3.763607,11.388020,0.84,12.249300,30.374,0.01068,0.11778,54794409,38968128,43.73,36.51,50.11,43.73,352676315136
3,AAPL,Apple Inc.,Technology,Consumer Electronics,Cupertino,CA,United States,100000,0.112,6.56,1.185531,7.536749,26.634148,0.88,29.047382,4.402,0.19875,1.45567,93465964,80546156,174.72,118.86,182.94,174.72,2851325607936
4,NVDA,NVIDIA Corporation,Technology,Semiconductors,Santa Clara,CA,United States,22473,0.528,6.78,1.411417,25.640358,40.843660,0.16,71.927280,10.619,0.17199,0.44832,52267288,57901618,276.92,127.00,346.47,276.92,690084642816
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,DLR,"Digital Realty Trust, Inc.",Real Estate,REIT—Office,Austin,TX,United States,3030,0.009,1.55,0.336122,8.862722,89.180650,4.88,58.300293,58.279,0.01308,0.09442,1625509,819189,138.23,130.10,178.22,138.23,39227875328
61,COF,Capital One Financial Corporation,Financial Services,Credit Services,McLean,VA,United States,50800,0.159,19.31,1.653319,1.823538,7.036769,2.40,5.081716,133.911,0.02959,0.20556,2992498,2240666,135.88,122.43,177.95,135.88,57833521152
62,BBY,"Best Buy Co., Inc.",Consumer Cyclical,Specialty Retail,Richfield,MN,United States,61200,0.005,9.23,1.587532,0.442666,10.433371,2.80,9.310645,17.547,0.10162,0.63223,3196866,1679729,96.30,85.58,141.97,96.30,23166023680
63,KMB,Kimberly-Clark Corporation,Consumer Defensive,Household & Personal Products,Dallas,TX,United States,45000,0.027,6.75,0.520669,2.083350,17.819260,4.64,20.448828,1.408,0.10050,2.29639,2023524,1860196,120.28,117.32,145.79,120.28,40500318208


In [22]:
info_df['industry'] = esg_data['peerGroup']

,Ticker,longName,sector,industry,city,state,country,fullTimeEmployees,revenueGrowth,forwardEps,beta,priceToSalesTrailing12Months,forwardPE,dividendRate,trailingPE,bookValue,returnOnAssets,returnOnEquity,averageVolume,volume,currentPrice,fiftyTwoWeekLow,fiftyTwoWeekHigh,regularMarketPrice,marketCap
0,INTC,Intel Corporation,Technology,Semiconductors,Santa Clara,CA,United States,121100,0.028,3.72,0.545767,2.670730,13.932797,1.46,10.664609,23.438,0.08586,0.22522,39453624,33249492,51.83,43.63,68.49,51.83,211051773952
1,MSFT,Microsoft Corporation,Technology,Software & Services,Redmond,WA,United States,181000,0.201,10.75,0.910925,12.312669,28.249302,2.48,32.344230,21.335,0.15249,0.49051,37450156,22566453,303.68,231.10,349.67,303.68,2276649336832
2,BAC,Bank of America Corporation,Financial Services,Banks,Charlotte,NC,United States,208000,0.125,3.84,1.415846,3.763607,11.388020,0.84,12.249300,30.374,0.01068,0.11778,54794409,38968128,43.73,36.51,50.11,43.73,352676315136
3,AAPL,Apple Inc.,Technology,Technology Hardware,Cupertino,CA,United States,100000,0.112,6.56,1.185531,7.536749,26.634148,0.88,29.047382,4.402,0.19875,1.45567,93465964,80546156,174.72,118.86,182.94,174.72,2851325607936
4,NVDA,NVIDIA Corporation,Technology,Semiconductors,Santa Clara,CA,United States,22473,0.528,6.78,1.411417,25.640358,40.843660,0.16,71.927280,10.619,0.17199,0.44832,52267288,57901618,276.92,127.00,346.47,276.92,690084642816
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,DLR,"Digital Realty Trust, Inc.",Real Estate,Real Estate,Austin,TX,United States,3030,0.009,1.55,0.336122,8.862722,89.180650,4.88,58.300293,58.279,0.01308,0.09442,1625509,819189,138.23,130.10,178.22,138.23,39227875328
61,COF,Capital One Financial Corporation,Financial Services,Diversified Financials,McLean,VA,United States,50800,0.159,19.31,1.653319,1.823538,7.036769,2.40,5.081716,133.911,0.02959,0.20556,2992498,2240666,135.88,122.43,177.95,135.88,57833521152
62,BBY,"Best Buy Co., Inc.",Consumer Cyclical,Retailing,Richfield,MN,United States,61200,0.005,9.23,1.587532,0.442666,10.433371,2.80,9.310645,17.547,0.10162,0.63223,3196866,1679729,96.30,85.58,141.97,96.30,23166023680
63,KMB,Kimberly-Clark Corporation,Consumer Defensive,Household Products,Dallas,TX,United States,45000,0.027,6.75,0.520669,2.083350,17.819260,4.64,20.448828,1.408,0.10050,2.29639,2023524,1860196,120.28,117.32,145.79,120.28,40500318208


In [27]:
# function to get the time based data from yahoo finance and storing it in a Dataframe
def create_date_based_dataframes(Symbols,start, end):
    # create empty dataframe
    stocks_df = pd.DataFrame()
    # iterate over each symbol
    for i in Symbols:  

        # print the symbol which is being downloaded
        print( str(Symbols.index(i)) + str(' : ') + i, sep=',', end=',', flush=True)  

        try:
            # download the stock price 
            stock = []
            stock = yf.download(i,start=start, end=end, progress=False)

            # append the individual stock prices 
            if len(stock) == 0:
                None
            else:
                stock['Ticker']=i
                stocks_df = stocks_df.append(stock,sort=False)
                print('data appended')
                
        except Exception:
            None
    # Data cleaning to ensure table names are proper, no null or duplicate data and datatypes are correct   
    stocks_df.dropna(inplace =True)
    stocks_df = stocks_df.rename_axis('Date').reset_index()
    stocks_df.insert(0, "Ticker", stocks_df.pop('Ticker'))
    return stocks_df

In [48]:
# Time series data start and end date YYYY,MM,DD
start = datetime.datetime(2018,3,16)
end = datetime.datetime(2022,3,16)
stock_historical_df = create_date_based_dataframes(Symbols,start, end)

0 : INTC,INTC: data appended
1 : MSFT,MSFT: data appended
2 : BAC,BAC: data appended
3 : AAPL,AAPL: data appended
4 : NVDA,NVDA: data appended
5 : VZ,VZ: data appended
6 : CSCO,CSCO: data appended
7 : EXC,EXC: data appended
8 : PEP,PEP: data appended
9 : MA,MA: data appended
10 : HIG,HIG: data appended
11 : C,C: data appended
12 : ANTM,ANTM: data appended
13 : ACN,ACN: data appended
14 : T,T: data appended
15 : IBM,IBM: data appended
16 : F,F: data appended
17 : OC,OC: data appended
18 : INTU,INTU: data appended
19 : WFC,WFC: data appended
20 : MRK,MRK: data appended
21 : JPM,JPM: data appended
22 : AEP,AEP: data appended
23 : UNH,UNH: data appended
24 : CI,CI: data appended
25 : AMAT,AMAT: data appended
26 : CL,CL: data appended
27 : EQIX,EQIX: data appended
28 : UPS,UPS: data appended
29 : NEM,NEM: data appended
30 : LMT,LMT: data appended
31 : BLK,BLK: data appended
32 : EIX,EIX: data appended
33 : JNJ,JNJ: data appended
34 : SYF,SYF: data appended
35 : FDX,FDX: data appended
36 : G

65

In [50]:
# Creating business business_controversy_df from esg_data to know more about business values lowering ESG scores

# Data cleaning to ensure table names are proper, no null or duplicate data and datatypes are correct   
business_controversy_df = esg_data['controversialBusinessAreas'].str.get_dummies(sep=', ')
business_controversy_df['Ticker'] = esg_data['Ticker']
business_controversy_df.insert(0, "Ticker", business_controversy_df.pop('Ticker'))
business_controversy_df.rename(columns={'Animal Testing': 'animalTesting', 'Catholic Values': 'catholicValues',
                                        'Controversial Weapons': 'controversialWeapons', 'Military Contracting': 'militaryContracting',
                                        'Small Arms': 'smallArms', 'Thermal Coal': 'thermalCoal'
                            }, inplace=True)


In [51]:
# Creating business controversy_type_df from esg_data to know more about controversies lowering ESG scores

# Data cleaning to ensure table names are proper, no null or duplicate data and datatypes are correct   
controversy_type_df = esg_data['relatedControversy'].str.get_dummies(sep=',')
controversy_type_df['Ticker'] = esg_data['Ticker']
controversy_type_df.insert(0, "Ticker", controversy_type_df.pop('Ticker'))
controversy_type_df.rename(columns={"Business Ethics Incidents": "businessEthicsIncidents", "Customer Incidents":"customerIncidents",
                                    "Employee Incidents":"employeeIncidents", "Environmental Supply Chain Incidents":"environmentalSupplyChainIncidents",
                                    "Governance Incidents" :"governanceIncidents","Operations Incidents":"operationsIncidents",
                                    "Product & Service Incidents" :"productServiceIncidents" , "Public Policy Incidents": "publicPolicyIncidents",
                                    "Social Supply Chain Incidents": "socialSupplyChainIncidents", "Society & Community Incidents" :"societyCommunityIncidents" 
                            }, inplace=True)
